In [5]:
pip install IPython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np

In [1]:
import pandas as pd
from datetime import datetime, timedelta

# ── CONFIG ───────────────────────────────────────────────────────
INPUT_FILE = r"C:\Users\hp\Downloads\GT1 Turbine Data Log Sheet for July 2025.xlsx"

DESCRIPTION = {
    "COMPRESSOR INLET PRESSURE": "CIP",
    "COMPRESSOR INLET AIR TEMP": "CIT",
    "COMPRESSOR DISCHARGE PRESSURE": "CDP",
    "COMPRESSOR DISCHARGE TEMPERATURE": "CDT",
    "IGV POSITION": "IGV_Position",
    "FILTER DIFFERENTIAL PRESSURE": "Filter_DP",
    "MAXIMUM VIBRATION": "VIB",
    "GT SPEED": "GT_Speed",
    "LOAD": "Load"
}

# Normalize lookup dict (upper case)
DESCRIPTION_NORMALIZED = {k.upper(): v for k, v in DESCRIPTION.items()}

# ── PROCESS ALL SHEETS ───────────────────────────────────────────
all_records = []

xls = pd.ExcelFile(INPUT_FILE)

for sheet_name in xls.sheet_names:
    try:
        sheet_date = pd.to_datetime(sheet_name.strip(), format="%d-%m-%Y", errors='coerce')
        if pd.isna(sheet_date):
            print(f"Skipped sheet (invalid date): {sheet_name}")
            continue
    except:
        print(f"Skipped sheet (error parsing date): {sheet_name}")
        continue

    # Read the sheet
    df_raw = pd.read_excel(xls, sheet_name=sheet_name, header=3)

    print(f"\n=== Sheet: {sheet_name} ===")
    print("Shape:", df_raw.shape)
    print("Columns (first 10):", df_raw.columns.tolist()[:10])

    # ── Check for DESCRIPTION column ───────────────────────────────
    if "DESCRIPTION" not in df_raw.columns:
        print("!!! CRITICAL: 'DESCRIPTION' column NOT FOUND in", sheet_name)
        print("   → Skipping this sheet")
        continue   # ← Now this is correct: skips to next sheet

    # If we reach here → DESCRIPTION column exists
    print("Found 'DESCRIPTION' column → processing...")

    # Clean DESCRIPTION column
    df_raw["DESCRIPTION"] = (
        df_raw["DESCRIPTION"]
        .astype(str)
        .str.strip()
        .str.upper()
    )

    # Filter only known parameters
    mask = df_raw["DESCRIPTION"].isin(DESCRIPTION_NORMALIZED.keys())
    df_params = df_raw[mask].copy()

    if df_params.empty:
        print(f"No matching parameters found in sheet: {sheet_name}")
        continue

    # Map to short names
    df_params["Parameter"] = df_params["DESCRIPTION"].map(DESCRIPTION_NORMALIZED)

    # Select hour columns (safely)
    if len(df_raw.columns) < 31:
        print(f"Warning: Only {len(df_raw.columns)} columns found → using from index 7 onward")
        hour_columns = df_params.columns[7:]
    else:
        hour_columns = df_params.columns[7:31]

    # Create records
    for _, row in df_params.iterrows():
        param = row["Parameter"]
        for i, col in enumerate(hour_columns):
            try:
                value = float(row[col])
                if pd.isna(value):
                    continue
            except (ValueError, TypeError):
                continue

            timestamp = sheet_date.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(hours=i)

            all_records.append({
                "Date": sheet_date.date(),
                "Timestamp": timestamp,
                "Parameter": param,
                "Value": value
            })
# ── Final DataFrame ──────────────────────────────────────────────
df_long = pd.DataFrame(all_records)

if df_long.empty:
    print("No valid data found across all sheets!")
else:
    # Sort for readability
    df_long = df_long.sort_values(["Parameter", "Timestamp"]).reset_index(drop=True)

    # Convert Value to numeric (just in case)
    df_long["Value"] = pd.to_numeric(df_long["Value"], errors='coerce')

    print(f"Total records: {len(df_long):,}")
    print(df_long.head(30))

    # Save long format (best for most analyses and ML)
    df_long.to_csv("GT1_Compressor_Long_Format_2025.csv", index=False)
    df_long.to_excel("GT1_Compressor_Long_Format_2025.xlsx", index=False)

    # Optional: Wide format (one column per parameter)
    df_wide = df_long.pivot_table(
        index="Timestamp",
        columns="Parameter",
        values="Value"
    ).reset_index()

    # Make sure all expected columns exist
    for param in DESCRIPTION.values():
        if param not in df_wide.columns:
            df_wide[param] = pd.NA

    # Order columns nicely
    df_wide = df_wide[['Timestamp'] + list(DESCRIPTION.values())]

    df_wide.to_csv("GT1_Compressor_Wide_Format_2025.csv", index=False)
    df_wide.to_excel("GT1_Compressor_Wide_Format_2025.xlsx", index=False)

    print("\nWide format preview:")
    print(df_wide.head())

Skipped sheet (invalid date): 01-07 -2025
Skipped sheet (invalid date): 02-0 7-2025 

=== Sheet: 03-07-2025  ===
Shape: (47, 31)
Columns (first 10): ['SUB SYSTEM', 'DESCRIPTION', 'KKS', 'UNIT', 'RANGE', 'ALARM', 'TRIP', datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0)]
Found 'DESCRIPTION' column → processing...

=== Sheet: 04-07-2025  ===
Shape: (47, 31)
Columns (first 10): ['SUB SYSTEM', 'DESCRIPTION', 'KKS', 'UNIT', 'RANGE', 'ALARM', 'TRIP', datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0)]
Found 'DESCRIPTION' column → processing...

=== Sheet: 05-07-2025  ===
Shape: (1048572, 41)
Columns (first 10): ['SUB SYSTEM', 'DESCRIPTION', 'KKS', 'UNIT', 'RANGE', 'ALARM', 'TRIP', datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0)]
Found 'DESCRIPTION' column → processing...

=== Sheet: 06-07-2025  ===
Shape: (47, 31)
Columns (first 10): ['SUB SYSTEM', 'DESCRIPTION', 'KKS', 'UNIT', 'RANGE', 'ALARM', 'TRIP', datetime.time(0, 0), datetime.time(1, 0), datetime.t

In [14]:
df_wide["CIP"] = df_wide["CIP"].dropna()
df_wide["CIP"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 281 entries, 0 to 280
Series name: CIP
Non-Null Count  Dtype  
--------------  -----  
281 non-null    float64
dtypes: float64(1)
memory usage: 2.3 KB


In [15]:
df_wide.to_csv("GT1_Compressor_Wide_Format_2025.csv", index=False)

In [37]:
result = pd.read_csv("GT1_Compressor_Wide_Format_2025.csv")

In [38]:
print(result)

               Timestamp    CIP   CIT   CDP    CDT  IGV_Position  Filter_DP  \
0    2025-07-03 00:00:00  128.6  24.0   9.0  335.0          63.0      171.3   
1    2025-07-03 01:00:00  124.2  24.0   9.0  333.0          61.0      171.2   
2    2025-07-03 02:00:00  123.3  24.0   8.0  328.0          58.0      168.8   
3    2025-07-03 03:00:00  123.3  24.0   8.0  327.0          57.0      166.9   
4    2025-07-03 17:00:00  101.2  25.0   8.0  331.0          57.0      137.1   
..                   ...    ...   ...   ...    ...           ...        ...   
276  2025-07-22 19:00:00  101.6  27.0  10.0  359.0          86.0        NaN   
277  2025-07-22 20:00:00  108.5  25.0  10.0  358.0          86.0        NaN   
278  2025-07-22 21:00:00  105.2  25.0   9.0  356.0          85.0        NaN   
279  2025-07-22 22:00:00  110.4  25.0  10.0  359.0          86.0        NaN   
280  2025-07-22 23:00:00  115.4  25.0  10.0  345.0          72.0        NaN   

     GT_Speed   Load  
0      3019.0   95.1  
1    

0       9.0
1       9.0
2       8.0
3       8.0
4       8.0
       ... 
276    10.0
277    10.0
278     9.0
279    10.0
280    10.0
Name: CDP, Length: 281, dtype: float64

In [40]:
result["Filter_DP"].dropna()

0      171.3
1      171.2
2      168.8
3      166.9
4      137.1
       ...  
252    161.5
253    164.1
254    121.5
255    167.5
256    164.7
Name: Filter_DP, Length: 257, dtype: float64